In [1]:
import sys
import os

sys.path.append('/import/silo2/aloe8475/Documents/edamame') #point to edamame locally
sys.path.append('/import/silo2/aloe8475/Documents/CODE/Analysis/Training/Associative Learning/EquilProp') #point to edamame locally

#Choose which nw:
nwChoice=700 #350 or 700

if nwChoice == 350:
    fileName='/import/silo2/aloe8475/Documents/Data/Associative Learning/Sparse Networks/asn_nw_00350_nj_01350_seed_1581_avl_10.00_disp_01.00_lx_50.00_ly_50.00'
elif nwChoice == 700:
    fileName='/import/silo2/aloe8475/Documents/Data/Associative Learning/Sparse Networks/asn_nw_00698_nj_02582_seed_002_avl_10.00_disp_01.00_lx_75.00_ly_75.00'

#point to network data path ^ 

saveFig='/import/silo2/aloe8475/Documents/Data/Figures/Equil Prop/' 
dataLoc='/import/silo2/aloe8475/Documents/Data/Associative Learning/EquilProp/'

#import edamame (neuromorphic nanowire python package by Ruomin Zhu)
from edamame import * 
import numpy as np
import matplotlib.pyplot as plt
import copy
import networkx as nx
from tqdm.notebook import tqdm_notebook as tqdm
from IPython.core.debugger import set_trace
import pandas as pd 

import pickle 
import _pickle as cPickle
import gzip


2022-05-31 12:46:42,461:INFO:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [ ]:
artLoc='/import/silo2/aloe8475/Documents/Data/Associative Learning/EquilProp/Sim Results/artemis/'
dataNoSim=[]

for b in tqdm(['b0p1','b0p2','b0p5','b2','b5']):
    saveName='698nw_2582nj_working_memory_nback_noReinforcement_7drains_data_'+b
    with open(artLoc+saveName+'.pkl', 'rb') as f:
        dataNoSim.append(pickle.load(f))

In [ ]:
#LOAD EXPERIMENTAL DATA:
loadLoc='/import/silo2/aloe8475/Documents/Data/Associative Learning/EquilProp/Adrian Results/'
fileName='non_reinforcement_09_00_02_PD.csv'
dataExp=pd.read_csv(loadLoc+fileName)

In [2]:
accuracy=np.zeros([len(dataNoSim),len(dataNoSim[0]),len(dataNoSim[0][0][3])])
for i in range(len(dataNoSim)):
    for j in range(len(dataNoSim[i])):
        accuracy[i][j]=dataNoSim[i][j][3]

NameError: name 'dataNoSim' is not defined

In [ ]:
nbackVals=np.zeros([len(dataNoSim),len(dataNoSim[0]),len(dataNoSim[0][0][3])])
for i in range(len(dataNoSim)):
    for j in range(len(dataNoSim[i])):
        nbackVals[i][j]=np.hstack(np.argwhere(np.array(dataNoSim[i][j][7])==-1)-np.argwhere(np.array(dataNoSim[i][j][7])==0))

In [ ]:
import pandas as pd
meanAcc=[];seAcc=[];numVals=[]
for n in range(len(nbackVals)):
    tmpMeanAcc=[];tmpSeAcc=[];tmpNumVals=[]
    for i in range(len(nbackVals[n])):
        tmp=nbackVals[n][i]
        tmpNum=[];tmpAcc=[]
        for j in range(1,nback+1):
            tmpNum.append(np.sum(tmp==j))
            tmpAcc.append(accuracy[n][i][tmp==j])
#             tmpSeAcc.append(np.nanstd(accuracy[n][i][tmp==j])/np.sqrt(len(accuracy[n][i][tmp==j])))
#         tmpAcc=np.hstack(tmpAcc)
        tmpMeanAcc.append(pd.DataFrame(tmpAcc).mean(axis=1).values)
        tmpSeAcc.append(pd.DataFrame(tmpAcc).std(axis=1).values/np.sqrt(len(tmpAcc)))
#         tmpNumVals.append(tmpNum)
    meanAcc.append(np.mean(tmpMeanAcc,axis=0))
    seAcc.append(np.std(tmpMeanAcc,axis=0)/np.sqrt(len(tmpMeanAcc)))
    numVals.append(tmpNumVals)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

plt.figure(figsize=(10,8))
nbacks=[1,2,3,4,5,6,7]#,8,9]
for i in range(len(meanAcc)):
    plt.plot(nbacks,meanAcc[i])
    plt.fill_between(nbacks,meanAcc[i]+seAcc[i],meanAcc[i]-seAcc[i],alpha=0.7)
plt.plot(nbacks,dataExp['Accuracy'],'--',c='k')
plt.xlabel('n-back')
plt.ylabel('Accuracy')
plt.legend(['b = 0.1','b = 0.2','b = 0.5','b = 2','b = 5','Experimental'])

# plt.savefig(saveFig+'698nw_2582nj_working_memory_no_reinforcement_nback7_b0p1_b5_wExperiment.pdf',format='pdf',dpi=300)